In [12]:
import pandas as pd
col_add = ["Experiment set",'Experiment id','subid', 'Thermal', 'Traffic noise levels', 'Speech']
master_data = pd.read_excel("../0_dataset_master/Experiment_edited_V3.1b.xlsx", usecols=["Start Date","Time_thermal"]+col_add)

new = master_data['Time_thermal'].str.split("-", n=1, expand=True)
master_data['New_Start_Date'] = pd.to_datetime(master_data['Start Date']).dt.date
master_data['New_End_Date'] = pd.to_datetime(master_data['Start Date']).dt.date
master_data['New_Start_Date'] = master_data['New_Start_Date'].astype(str) + ' ' + new[0]
master_data['New_End_Date'] = master_data['New_End_Date'].astype(str) + ' ' + new[1]
master_data['New_Start_Date'] = master_data['New_Start_Date'].astype('datetime64[ms]')
master_data['New_End_Date'] = master_data['New_End_Date'].astype('datetime64[ms]')

In [13]:
data_set = {}
new_set = [6]
ID_arr = [1,2,3,4,5,6]
try:
    for set_ in new_set:
        data_set["set"+str(set_)]={}
        for id_ in ID_arr:
            data_set["set"+str(set_)]["id"+str(id_)] = pd.read_excel(f"SkinT_set{set_}.xlsx", sheet_name=f"id{id_}", parse_dates=['Time'], date_format="mixed")
except ValueError:
    print(set_, id_)

In [3]:
# NEED TO REFINE cause of inconsistent date-format
from datetime import datetime
refineset = ["set1","set2"]
for set_ in refineset:
    for id_ in ID_arr:
        date_result = []
        # print(f"process on {set_} id{id_}")
        for _, row in data_set[set_]["id"+str(id_)].iterrows():
            date_ = row['Time']
            if date_.day == 13:
                date_result.append(date_)
                continue    
            date_ = datetime.strptime(str(date_), "%Y-%d-%m %H:%M:%S")
            date_result.append(date_)
        data_set[set_]["id"+str(id_)].Time=date_result

KeyError: 'set1'

In [15]:
from datetime import datetime, timedelta
for set_ in data_set.keys():
    with pd.ExcelWriter(f"SkinT_{set_}_adjustment.xlsx", engine="xlsxwriter") as writer:
        for id_ in data_set[set_].keys():
            print(f"process on {set_} {id_}")
            temp_df = data_set[set_][id_]

            slice_data = master_data[(master_data['Experiment set']==f'{set_}') & (master_data['Experiment id'] == f'{id_}')]
            # display(slice_data)
            resTemp = []

            for index, row in temp_df.iterrows():
                # print(row)
                delta = 0
                max_timestamp = row['Time'].replace(hour=23, minute=59, second=59)
                # timestamp = datetime.strptime(str(row['Time']), "%Y-%m-%d %H:%M:%S")
                timestamp = row['Time']
                # print(timestamp.month)
                while True:
                    timestamp = timestamp + timedelta(minutes=delta)
                    # display(slice_data['New_Start_Date'])
                    sliced = slice_data[(slice_data['New_Start_Date']<=timestamp) & (slice_data['New_End_Date']>=timestamp)]
                    # print(f"Slice data not found: {timestamp}")
                    # break
                    if timestamp > max_timestamp:
                        # print(f"Slice data not found: {row['Time']}")
                        resTemp.append([pd.NA]*len(col_add))
                        break

                    if sliced.empty:
                        delta+=5
                        continue
                    # print(sliced)
                    resTemp.append(list(sliced[col_add].values[0]))
                    break
        #         break
        #     break
        # break
            
            temp_df[col_add] = resTemp
            # display(temp_df)
            # break
        # break

            temp_df.to_excel(writer, sheet_name=f"id{id_}", index=False)

process on set6 id1
process on set6 id2
process on set6 id3
process on set6 id4
process on set6 id5
process on set6 id6
